In [209]:
import pandas as pd
import psycopg2
import traceback

from datetime import datetime

date = datetime.now().strftime('%d-%m-%Y')

# Funciones

In [62]:
def start():
    connection = psycopg2.connect(dbname = 'Carriers',
                    host = '188.166.63.213',
                    port = '5432',
                    user = 'cperez',
                    password = 'srb4o7SlqAIf')

    return connection

In [115]:
def query_db (query):
    try:
        connection = start()
        with connection.cursor() as cursor:
            cursor.execute(query)
            result = cursor.fetchall()
        connection.commit()
        return result
    except Exception as ex:
        traceback.print_exc(ex)
    finally:
        connection.close()


In [37]:
def extractCarrier(df, carrier_name, line_of_business, carrier_column = 'Carrier'):
    df =  df[(df[carrier_column] == carrier_name) & (df['Line of Business'] == line_of_business)]
    return df

In [42]:
def leftJoinBOB(new_policies, bob_policies):
    upload = list(set(new_policies) - set(bob_policies) )
    return upload

# Aetna

In [65]:
connection = start()

In [14]:
df_a = pd.read_excel('/mnt/c/Users/cesar/Downloads/EnrollmentRoster.xlsx').drop(columns= 'Unnamed: 0')

In [15]:
df_a.columns

Index(['Member ID', 'Legacy Member ID', 'Medicare\nNumber',
       'Affinity Policy ID', 'First Name', 'Mid Initial', 'Last Name',
       'Date of Birth', 'Phone Number', 'Address Line 1 ', 'Address Line 2',
       'City', 'State', 'Zip Code ', 'App Sign Date', 'App Rec Date',
       'Cov Eff Date', 'Mbr Status', 'Term Date', 'Term Rsn Code', 'Plan Name',
       'Plan Eff Date', 'Supplement Plans ', 'Writing Agent NPN',
       'Writing Agent TIN', 'Writing Agent First Name',
       'Writing Agent Last Name', 'Pay Method', 'Bill Option', 'Invoice Amt',
       'Invce Date', 'CMS Contract Nbr', 'PBP Code', 'ID Card Issue Date',
       'Affnty Code', 'Producer Level  1 Name', 'Producer Level 1 TIN/NPN',
       'Prdcr Level 1 Code', 'Producer Level  2 Name',
       'Producer Level 2 TIN/NPN', 'Prdcr Level  2 Code',
       'Producer Level  3 Name', 'Producer Level 3 TIN/NPN',
       'Prdcr Level  3 Code', 'Producer Level  4 Name',
       'Producer Level 4 TIN/NPN', 'Prdcr Level  4 Code',
   

## BOB SalesForce

In [39]:
df_b = pd.read_csv('//mnt/c/Users/cesar/Desktop/claro/bob/report1674816107684.csv', encoding='latin-1', low_memory=False)
df_b.head(3)

,Book of Business: BOB Name,Account Executive Name,Carrier,NPN AOR,Agent AOR,Internal Agent,NPN Internal Agent,Contact,Policy Number,Source,...,Email,Metal Level,Zip Code,Book of Business: Last Modified Date,Note,Claro Type,Previous BOB ID,Carrier State line of Business,Book of Business: ID,Account Executive
0,BOB# 00000001,Miguel Bautista,Molina,1647487.0,Amarilys Perez Lazo,Amarilys Perez Lazo,1647487.0,Amarilys Perez Lazo,FLE192951620161,Direct,...,NaN,Silver,NaN,12/20/2022,NaN,Renewals,NaN,Molina Florida,a1C1G000007eLyj,
1,BOB# 00000002,Miguel Bautista,Molina,1647487.0,Amarilys Perez Lazo,Amarilys Perez Lazo,1647487.0,Amarilys Perez Lazo,FLE192921397071,Direct,...,NaN,Bronze,NaN,12/20/2022,NaN,Renewals,NaN,Molina Florida,a1C1G000007eLyk,
2,BOB# 00000003,Miguel Bautista,Molina,1647487.0,AMARILYS PEREZ LAZO,Amarilys Perez Lazo,1647487.0,Amarilys Perez Lazo,FLE192921385481,Direct,...,NaN,Bronze,NaN,12/20/2022,V-ActualizaciÃ³n 09-01,Renewals,NaN,Molina Florida,a1C1G000007eLyl,


## Homologacion

In [20]:
df_template = pd.read_csv('//mnt/c/Users/cesar/Desktop/claro/Format_Create_BOB.csv').head(0)
df_template.columns

Index(['Carrier___', 'LOB____', 'Carrier', 'Agent AOR', 'NPN AOR',
       'Contact____', 'NPN_Internal____', 'Contact', 'Policy Number', 'Source',
       'Referrals_____', 'Referrals Id', 'Status', 'Effective Date',
       'Transaction Date', 'Policy Term Date', 'Member First Name',
       'Member Last Name', 'DOB', 'State', 'County', 'City', 'Members',
       'Policy Type', 'Last Status Modification Date', 'Loading Date',
       'Record Type Id', 'Plan', 'Election Period', 'Final Premium', 'Subsidy',
       'Annual Premium', 'Paid Thru Date', 'Days Past Due', 'Phone Number',
       'Email', 'Metal Level', 'Zip Code', 'Claro Type',
       'Carrier State line of Business'],
      dtype='object')

In [21]:
df_a = df_a.rename(
    columns={
        'Member ID':'PolicyNumber',
        'Writing Agent NPN':'NPN_AOR',
        'Mbr Status':'Status',
        'Plan Eff Date':'EffectiveDate',
        'App Sign Date':'Transaction Date',
        'Term Date':'Policy Term Date',        
    }
)

In [22]:
df_a.columns

Index(['PolicyNumber', 'Legacy Member ID', 'Medicare\nNumber',
       'Affinity Policy ID', 'First Name', 'Mid Initial', 'Last Name',
       'Date of Birth', 'Phone Number', 'Address Line 1 ', 'Address Line 2',
       'City', 'State', 'Zip Code ', 'Transaction Date', 'App Rec Date',
       'Cov Eff Date', 'Status', 'Policy Term Date', 'Term Rsn Code',
       'Plan Name', 'EffectiveDate', 'Supplement Plans ', 'NPN_AOR',
       'Writing Agent TIN', 'Writing Agent First Name',
       'Writing Agent Last Name', 'Pay Method', 'Bill Option', 'Invoice Amt',
       'Invce Date', 'CMS Contract Nbr', 'PBP Code', 'ID Card Issue Date',
       'Affnty Code', 'Producer Level  1 Name', 'Producer Level 1 TIN/NPN',
       'Prdcr Level 1 Code', 'Producer Level  2 Name',
       'Producer Level 2 TIN/NPN', 'Prdcr Level  2 Code',
       'Producer Level  3 Name', 'Producer Level 3 TIN/NPN',
       'Prdcr Level  3 Code', 'Producer Level  4 Name',
       'Producer Level 4 TIN/NPN', 'Prdcr Level  4 Code',
    

## Limpiar las polizas 

In [45]:
df_aetna = df_a.dropna(subset=['PolicyNumber'])

## Extraer Polizas de Aetna BOB

In [40]:
df_b['Line of Business'].unique()

array(['ACA', 'Medicare', 'Life', 'Supplementary'], dtype=object)

In [41]:
df_medicare = extractCarrier(df_b, 'Aetna', 'Medicare')

## Extract New Policies

In [46]:
upload = leftJoinBOB(df_aetna['PolicyNumber'].values, df_medicare['Policy Number'].values)

In [52]:
df_upload = df_aetna[df_aetna['PolicyNumber'].isin(upload)]
df_upload.head(3)

,PolicyNumber,Legacy Member ID,Medicare\nNumber,Affinity Policy ID,First Name,Mid Initial,Last Name,Date of Birth,Phone Number,Address Line 1,...,Producer Level 6 TIN/NPN,Prdcr Level 6 Code,Producer Level 7 Name,Producer Level 7 TIN/NPN,Prdcr Level 7 Code,Producer Level 8 Name,Producer Level 8 TIN/NPN,Prdcr Level 8 Code,Producer Level 9 Name,Producer Level 9 TIN/NPN
3,101687516900,NaN,6MQ3NP3XE84,T123001853925A,N,NaN,Gonzalez Martinez,1956-08-28,4.079549e+09,237 BALBOA DR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101691065800,NaN,8K97GE4UR71,T123002660986A,THELMA,NaN,HERNANDEZ,1940-07-14,7.864396e+09,7049 CARNA CT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,101689237700,NaN,1U81GW9MK15,CS90HWG1G0,CANDIDA,R,MONTANEZ,1946-09-04,6.892452e+09,2806 EAGLE CLAW CT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Upload into template

In [50]:
df_upload.reset_index(drop=True, inplace=True)

In [170]:
df_upload.reset_index(inplace=True)

In [51]:
df_template = pd.read_csv('//mnt/c/Users/cesar/Desktop/claro/Format_Create_BOB.csv').head(0)
df_template

,Carrier___,LOB____,Carrier,Agent AOR,NPN AOR,Contact____,NPN_Internal____,Contact,Policy Number,Source,...,Subsidy,Annual Premium,Paid Thru Date,Days Past Due,Phone Number,Email,Metal Level,Zip Code,Claro Type,Carrier State line of Business


In [54]:
df_upload.columns

Index(['PolicyNumber', 'Legacy Member ID', 'Medicare\nNumber',
       'Affinity Policy ID', 'First Name', 'Mid Initial', 'Last Name',
       'Date of Birth', 'Phone Number', 'Address Line 1 ', 'Address Line 2',
       'City', 'State', 'Zip Code ', 'Transaction Date', 'App Rec Date',
       'Cov Eff Date', 'Status', 'Policy Term Date', 'Term Rsn Code',
       'Plan Name', 'EffectiveDate', 'Supplement Plans ', 'NPN_AOR',
       'Writing Agent TIN', 'Writing Agent First Name',
       'Writing Agent Last Name', 'Pay Method', 'Bill Option', 'Invoice Amt',
       'Invce Date', 'CMS Contract Nbr', 'PBP Code', 'ID Card Issue Date',
       'Affnty Code', 'Producer Level  1 Name', 'Producer Level 1 TIN/NPN',
       'Prdcr Level 1 Code', 'Producer Level  2 Name',
       'Producer Level 2 TIN/NPN', 'Prdcr Level  2 Code',
       'Producer Level  3 Name', 'Producer Level 3 TIN/NPN',
       'Prdcr Level  3 Code', 'Producer Level  4 Name',
       'Producer Level 4 TIN/NPN', 'Prdcr Level  4 Code',
    

In [55]:
df_template['Member First Name'] = df_upload['First Name']
df_template['Member Last Name'] = df_upload['Last Name']

In [56]:
df_template.reset_index(drop=True, inplace=True)

In [57]:
df_template['Policy Number'] = df_upload['PolicyNumber']

In [58]:
df_template['Plan'] = df_upload['Plan Name']

In [111]:
query_agebt_aor = """
    SELECT contact, agent_writing_number
    FROM agent_contract 
    WHERE npn = \'{npn_aor}\' ;
"""

In [137]:
npn_list = pd.to_numeric(df_upload['NPN_AOR'], downcast='integer')

In [127]:
box_contact = []
for i in npn_list:
    df = query_db(query_agebt_aor.format(npn_aor=int(i)))
    df = pd.DataFrame(df, columns=['Contact', 'awn'])
    assert df['Contact'].unique().shape[0] == 1, 'Error en la integridad de los datos en base de datos, por favor consultar el agente con el NPN {}'.format(i)
    box_contact.append(df['Contact'].unique()[0])

In [129]:
df_template['Agent AOR'] = box_contact

In [138]:
df_template['NPN AOR'] = df_upload['NPN_AOR']

In [139]:
df_template['Effective Date'] = df_upload['EffectiveDate']

In [142]:
df_template['Transaction Date'] = df_upload['Transaction Date']

In [144]:
df_template['Policy Term Date'] = df_upload['Policy Term Date']

In [145]:
df_template['LOB____'] = 'Medicare'

### Source

In [146]:
def assign_source(referal_agents:list, df_template, column_agent_numbers:str, df_upload) -> pd.DataFrame:
    df_s = df_template.copy()
    mask_s = df_upload[column_agent_numbers].isin(referal_agents)
    
    df_s.loc[mask_s,  'Source'] = 'Referal'
    df_s.loc[~mask_s, 'Source'] = 'Direct'
    
    return df_s

In [154]:
npn_referals = [18233171, 7388594, 19603471, 18522178, 19106367, 16001347, 10041154, 20462802, 17686050]

#npn_referals = str([str(i)  for i in npn_referals]).replace('[', '').replace(']','')


In [ ]:
df_s = df_template.copy()
mask_s = df_upload[column_agent_numbers].isin(referal_agents)
    
df_s.loc[mask_s,  'Source'] = 'Referal'
df_s.loc[~mask_s, 'Source'] = 'Direct'

In [177]:
df_template = assign_source(npn_referals, df_template, 'NPN_AOR', df_upload)

In [179]:
df_template['Status'] = df_upload['Status']

In [181]:
df_template['State'] = df_upload['State']

In [182]:
query_record_type = """
    SELECT  id  FROM record_type rt
	where record_type = 'BOB';
"""

In [185]:
df_template['Record Type Id'] = query_db(query_record_type)[0][0]

## Carrier

In [205]:
def exrtact_carrier_id(serie_state, carrier_name, lob):
    box = []
    query_carrier_id = """
    SELECT carrier_id  
    FROM carrier c 
    WHERE carrier = '{carrier_name}' and state_c = '{carrier_state}' and lob = '{lob}'
    ;
"""
    
    for state in serie_state:
        carrier_id = query_db(query_carrier_id.format(carrier_name=carrier_name, carrier_state=state, lob=lob))
        if len(carrier_id) == 1:
            box.append(carrier_id[0][0])
        else:
            box.append(None)
    
    return box

In [207]:
df_template['Carrier'] =  exrtact_carrier_id(df_template['State'], 'Aetna', 'Medicare')

In [210]:
df_template['Last Status Modification Date'] = date

In [211]:
df_template['Loading Date'] = date

In [212]:
query_cslb = """
    select carrier_lob_id 
    from carrier c
    where carrier_id = '{carrier_id}' and state_c = '{state}';
"""

In [ ]:
def making_query_cslb(df_template, query_cslb):
    box = []
    carrier = df_template['Carrier'].unique()
    
    for state in df_template.State.values:
    	query_cslb = query_cslb.format(carrier_id=carrier, state=state[0])
    value = query_db(query_cslb)[0][0]
    return value

In [ ]:
def extract_cslb(df_template, query_cslb):
    cslb = making_query_cslb(df_template, query_cslb)
    df_template['Carrier State line of Business'] = cslb
    
    return df_template

In [ ]:
df_template = extract_cslb(df_template, query_cslb)

In [140]:
df_upload.columns

Index(['PolicyNumber', 'Legacy Member ID', 'Medicare\nNumber',
       'Affinity Policy ID', 'First Name', 'Mid Initial', 'Last Name',
       'Date of Birth', 'Phone Number', 'Address Line 1 ', 'Address Line 2',
       'City', 'State', 'Zip Code ', 'Transaction Date', 'App Rec Date',
       'Cov Eff Date', 'Status', 'Policy Term Date', 'Term Rsn Code',
       'Plan Name', 'EffectiveDate', 'Supplement Plans ', 'NPN_AOR',
       'Writing Agent TIN', 'Writing Agent First Name',
       'Writing Agent Last Name', 'Pay Method', 'Bill Option', 'Invoice Amt',
       'Invce Date', 'CMS Contract Nbr', 'PBP Code', 'ID Card Issue Date',
       'Affnty Code', 'Producer Level  1 Name', 'Producer Level 1 TIN/NPN',
       'Prdcr Level 1 Code', 'Producer Level  2 Name',
       'Producer Level 2 TIN/NPN', 'Prdcr Level  2 Code',
       'Producer Level  3 Name', 'Producer Level 3 TIN/NPN',
       'Prdcr Level  3 Code', 'Producer Level  4 Name',
       'Producer Level 4 TIN/NPN', 'Prdcr Level  4 Code',
    